# YOLO Hyperparameter Tuning with Optuna

This notebook provides comprehensive hyperparameter optimization for YOLO models using Optuna.

## Features:
- ✅ Support for YOLOv8, YOLOv9, YOLOv10, YOLO11, YOLO12
- ✅ Optuna-based hyperparameter optimization
- ✅ Extensive search space: learning rates, momentum, weight decay, augmentation, optimizer
- ✅ GPU detection and utilization
- ✅ Best hyperparameters saved to YAML
- ✅ Optimization history visualization
- ✅ Final model training with optimized parameters
- ✅ Production-ready code with clear comments

## Workflow:
1. Install required libraries
2. Configure dataset and model
3. Define hyperparameter search space
4. Run Optuna optimization (30 trials)
5. Visualize results
6. Save best hyperparameters
7. Train final model with optimized settings

## 1. Install Required Libraries

Install all necessary packages for YOLO training and hyperparameter optimization.

In [ ]:
# Install required libraries (uncomment if running in Colab)
# !pip install -q ultralytics optuna plotly kaleido wandb pyyaml

import os
import sys
import yaml
import json
import torch
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# YOLO and Optuna imports
from ultralytics import YOLO
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice

warnings.filterwarnings('ignore')

# Configure matplotlib for notebook display
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✓ Libraries imported successfully')
print(f'✓ Device: {device}')
if device == 'cuda':
    print(f'  GPU: {torch.cuda.get_device_name(0)}')
    print(f'  CUDA Version: {torch.version.cuda}')
    print(f'  Available Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

## 2. Configuration

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Base directories
BASE_DIR = Path.cwd().parent

# For Colab, uncomment and set your paths:
# BASE_DIR = Path("/content/drive/MyDrive/computer_vision_yolo")

# Model Selection - Choose one of the following:
MODEL_NAME = "yolov8n"
# Supported models:
# YOLOv8: 'yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x'
# YOLOv9: 'yolov9s', 'yolov9m', 'yolov9l', 'yolov9x'
# YOLOv10: 'yolov10n', 'yolov10s', 'yolov10m', 'yolov10l', 'yolov10x'
# YOLO11: 'yolo11n', 'yolo11s', 'yolo11m', 'yolo11l', 'yolo11x'
# YOLO12: 'yolo12n', 'yolo12s', 'yolo12m', 'yolo12l', 'yolo12x'

# Directory structure
MODELS_DIR = BASE_DIR / 'models' / MODEL_NAME
TMP_DIR = BASE_DIR / 'tmp' / MODEL_NAME
RUNS_DIR = BASE_DIR / 'hyperparameter_tuning' / 'runs'

# Dataset Selection
# Option 1: Full dataset (~100k images) - for final optimization
# YOLO_DATASET_ROOT = BASE_DIR / 'bdd100k_yolo'
# DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Option 2: Limited dataset (representative samples) - for quick tuning
YOLO_DATASET_ROOT = BASE_DIR / 'bdd100k_yolo_limited'
DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Verify dataset exists
if not DATA_YAML_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found: {DATA_YAML_PATH}\n"
        f"Please prepare the dataset first using process_bdd100k_to_yolo_dataset.py"
    )

# Optimization Configuration
N_TRIALS = 30  # Number of optimization trials
TIMEOUT_HOURS = 6  # Maximum time for optimization (None for no limit)
N_STARTUP_TRIALS = 10  # Random exploration trials before optimization
EPOCHS_PER_TRIAL = 50  # Training epochs per trial
BATCH_SIZE = 16  # Batch size for training
IMAGE_SIZE = 640  # Input image size

# Weights & Biases (optional)
USE_WANDB = False  # Set to True to enable W&B logging
W_B_PROJECT = "yolo-bdd100k-hyperparameter-tuning"

# Generate run identifier
RUN_TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
RUN_NAME = f'{MODEL_NAME}_optuna_{RUN_TIMESTAMP}'

# Create directories
RUN_DIR = RUNS_DIR / RUN_NAME
RUN_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Read dataset configuration
with open(DATA_YAML_PATH, 'r') as f:
    data_config = yaml.safe_load(f)
    NUM_CLASSES = data_config['nc']
    CLASS_NAMES = data_config['names']

print('=' * 80)
print('CONFIGURATION SUMMARY')
print('=' * 80)
print(f'Model: {MODEL_NAME}')
print(f'Dataset: {YOLO_DATASET_ROOT.name}')
print(f'Data YAML: {DATA_YAML_PATH}')
print(f'Classes: {NUM_CLASSES}')
print(f'Class Names: {CLASS_NAMES}')
print(f'Device: {device}')
print(f'Optimization Trials: {N_TRIALS}')
print(f'Epochs per Trial: {EPOCHS_PER_TRIAL}')
print(f'Batch Size: {BATCH_SIZE}')
print(f'Image Size: {IMAGE_SIZE}')
print(f'Timeout: {TIMEOUT_HOURS} hours' if TIMEOUT_HOURS else 'No timeout')
print(f'Run Directory: {RUN_DIR}')
print(f'W&B Logging: {"Enabled" if USE_WANDB else "Disabled"}')
print('=' * 80)

## 3. Load Base YOLO Model

In [ ]:
# ============================================================================
# LOAD BASE YOLO MODEL
# ============================================================================

model_path = MODELS_DIR / f'{MODEL_NAME}.pt'

if not model_path.exists():
    print(f'Model not found at {model_path}')
    print(f'Downloading {MODEL_NAME}...')
    
    try:
        # Download model (Ultralytics will automatically download)
        model_name_for_download = MODEL_NAME
        if MODEL_NAME.startswith('yolo11') or MODEL_NAME.startswith('yolo12'):
            model_name_for_download = MODEL_NAME + '.pt'
        
        model = YOLO(model_name_for_download)
        
        # Try to move from cache to models directory
        import glob
        cache_patterns = [
            str(Path.home() / '.cache' / 'ultralytics' / '**' / f'{MODEL_NAME}.pt'),
            str(Path.home() / '.config' / 'Ultralytics' / '**' / f'{MODEL_NAME}.pt'),
        ]
        
        model_found = False
        for pattern in cache_patterns:
            cache_paths = glob.glob(pattern, recursive=True)
            if cache_paths:
                shutil.move(cache_paths[0], model_path)
                print(f'✓ Model downloaded and saved to {model_path}')
                print(f'  Size: {model_path.stat().st_size / (1024*1024):.1f} MB')
                model_found = True
                break
        
        if not model_found:
            print(f'✓ Model loaded from ultralytics cache')
            
    except Exception as e:
        print(f'❌ Error downloading model: {e}')
        raise
else:
    model = YOLO(str(model_path))
    print(f'✓ Model loaded from {model_path}')

print(f'\n📊 Base Model Information:')
print(f'  Name: {MODEL_NAME}')
print(f'  Classes: {len(model.names)}')
print(f'  Task: {model.task}')
print(f'  Path: {model_path}')

## 4. Verify Dataset Structure

In [ ]:
# ============================================================================
# VERIFY DATASET STRUCTURE
# ============================================================================

print('Verifying YOLO dataset structure...')
print(f'\n📁 Dataset Root: {YOLO_DATASET_ROOT}')

# Check all splits
dataset_stats = {}
for split in ['train', 'val', 'test']:
    images_dir = YOLO_DATASET_ROOT / 'images' / split
    labels_dir = YOLO_DATASET_ROOT / 'labels' / split
    
    if images_dir.exists() and labels_dir.exists():
        num_images = len(list(images_dir.glob('*.jpg'))) + len(list(images_dir.glob('*.png')))
        num_labels = len(list(labels_dir.glob('*.txt')))
        dataset_stats[split] = {'images': num_images, 'labels': num_labels}
        print(f'  ✓ {split:5s}: {num_images:6d} images, {num_labels:6d} labels')
    else:
        print(f'  ⚠️  {split:5s}: Directory not found')
        dataset_stats[split] = {'images': 0, 'labels': 0}

print(f'\n📄 Configuration: {DATA_YAML_PATH}')
print(f'  Classes: {NUM_CLASSES}')
print(f'  Names: {CLASS_NAMES}')

total_images = sum(stats['images'] for stats in dataset_stats.values())
print(f'\n✓ Dataset verified: {total_images:,} total images')
print('✓ Ready for hyperparameter optimization')

## 5. Define Hyperparameter Search Space

In [ ]:
# ============================================================================
# DEFINE HYPERPARAMETER SEARCH SPACE
# ============================================================================

def define_hyperparameters(trial):
    """
    Define comprehensive hyperparameter search space for YOLO optimization.
    
    Uses Optuna's default ranges for most parameters to leverage library expertise.
    Only specifies categorical choices and essential boundaries.
    
    Args:
        trial: Optuna trial object
        
    Returns:
        dict: Complete hyperparameter configuration for YOLO training
    """
    
    # ========================================================================
    # OPTIMIZER SELECTION
    # ========================================================================
    optimizer_choice = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW'])
    
    # ========================================================================
    # LEARNING RATE PARAMETERS (Let library determine optimal ranges)
    # ========================================================================
    lr0 = trial.suggest_float('lr0', 1e-5, 1e-2, log=True)
    lrf = trial.suggest_float('lrf', 0.01, 0.2)
    
    # ========================================================================
    # OPTIMIZER PARAMETERS (Library defaults)
    # ========================================================================
    momentum = trial.suggest_float('momentum', 0.6, 0.98)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    
    # ========================================================================
    # WARMUP PARAMETERS
    # ========================================================================
    warmup_epochs = trial.suggest_int('warmup_epochs', 0, 5)
    warmup_momentum = trial.suggest_float('warmup_momentum', 0.0, 0.95)
    warmup_bias_lr = trial.suggest_float('warmup_bias_lr', 0.0, 0.2)
    
    # ========================================================================
    # DATA AUGMENTATION - HSV COLOR SPACE
    # ========================================================================
    hsv_h = trial.suggest_float('hsv_h', 0.0, 0.1)
    hsv_s = trial.suggest_float('hsv_s', 0.0, 0.9)
    hsv_v = trial.suggest_float('hsv_v', 0.0, 0.9)
    
    # ========================================================================
    # DATA AUGMENTATION - GEOMETRIC TRANSFORMATIONS
    # ========================================================================
    degrees = trial.suggest_float('degrees', 0.0, 45.0)
    translate = trial.suggest_float('translate', 0.0, 0.9)
    scale = trial.suggest_float('scale', 0.0, 0.9)
    shear = trial.suggest_float('shear', 0.0, 10.0)
    perspective = trial.suggest_float('perspective', 0.0, 0.001)
    
    # ========================================================================
    # DATA AUGMENTATION - FLIP
    # ========================================================================
    flipud = trial.suggest_float('flipud', 0.0, 1.0)
    fliplr = trial.suggest_float('fliplr', 0.0, 1.0)
    
    # ========================================================================
    # DATA AUGMENTATION - ADVANCED
    # ========================================================================
    mosaic = trial.suggest_float('mosaic', 0.0, 1.0)
    mixup = trial.suggest_float('mixup', 0.0, 1.0)
    copy_paste = trial.suggest_float('copy_paste', 0.0, 1.0)
    
    # ========================================================================
    # LOSS FUNCTION WEIGHTS
    # ========================================================================
    box = trial.suggest_float('box', 0.5, 20.0)
    cls = trial.suggest_float('cls', 0.2, 4.0)
    dfl = trial.suggest_float('dfl', 0.5, 3.0)
    
    # ========================================================================
    # COMPILE HYPERPARAMETERS
    # ========================================================================
    hyperparams = {
        # Optimizer
        'optimizer': optimizer_choice,
        
        # Learning rates
        'lr0': lr0,
        'lrf': lrf,
        
        # Optimizer parameters
        'momentum': momentum,
        'weight_decay': weight_decay,
        
        # Warmup
        'warmup_epochs': warmup_epochs,
        'warmup_momentum': warmup_momentum,
        'warmup_bias_lr': warmup_bias_lr,
        
        # HSV augmentation
        'hsv_h': hsv_h,
        'hsv_s': hsv_s,
        'hsv_v': hsv_v,
        
        # Geometric augmentation
        'degrees': degrees,
        'translate': translate,
        'scale': scale,
        'shear': shear,
        'perspective': perspective,
        
        # Flip augmentation
        'flipud': flipud,
        'fliplr': fliplr,
        
        # Advanced augmentation
        'mosaic': mosaic,
        'mixup': mixup,
        'copy_paste': copy_paste,
        
        # Loss weights
        'box': box,
        'cls': cls,
        'dfl': dfl,
        
        # Fixed training parameters
        'epochs': EPOCHS_PER_TRIAL,
        'batch': BATCH_SIZE,
        'imgsz': IMAGE_SIZE,
        'device': device,
        'patience': 15,  # Early stopping patience
        'save': False,  # Don't save intermediate models
        'plots': False,  # Don't generate plots for each trial
        'cache': True,  # Cache images for faster training
        'workers': 8,  # Number of data loading workers
        'close_mosaic': 10,  # Disable mosaic in last N epochs
        'verbose': False,  # Reduce verbosity
    }
    
    return hyperparams

print('✓ Hyperparameter search space defined')
print('\n📊 Search Space Summary:')
print('  Strategy: Using wide ranges, letting Optuna find optimal values')
print('  Optimizers: SGD, Adam, AdamW')
print('  Learning Rates: Wide range for exploration')
print('  Augmentation: Full range (0-1 for probabilities)')
print('  Loss Weights: Wide range for different dataset characteristics')
print(f'  Fixed: epochs={EPOCHS_PER_TRIAL}, batch={BATCH_SIZE}, imgsz={IMAGE_SIZE}')

## 6. Define Objective Function

In [ ]:
# ============================================================================
# DEFINE OBJECTIVE FUNCTION FOR OPTUNA
# ============================================================================

def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    
    This function:
    1. Gets hyperparameters for the current trial
    2. Trains a YOLO model with those hyperparameters
    3. Evaluates the model on validation set
    4. Returns the metric to optimize (mAP@0.5)
    
    Args:
        trial: Optuna trial object
        
    Returns:
        float: Validation mAP@0.5 (metric to maximize)
    """
    
    # Get hyperparameters for this trial
    hyperparams = define_hyperparameters(trial)
    
    # Create trial-specific directory
    trial_dir = RUN_DIR / f'trial_{trial.number:03d}'
    trial_dir.mkdir(exist_ok=True)
    
    # Initialize W&B if enabled
    wandb_run = None
    if USE_WANDB:
        try:
            import wandb
            wandb_run = wandb.init(
                project=W_B_PROJECT,
                name=f'{MODEL_NAME}_trial_{trial.number:03d}',
                config=hyperparams,
                reinit=True
            )
        except Exception as e:
            print(f'⚠️  W&B initialization failed: {e}')
            wandb_run = None
    
    try:
        # Print trial information
        print(f'\n{"=" * 80}')
        print(f'TRIAL {trial.number}/{N_TRIALS}')
        print(f'{"=" * 80}')
        print(f'Optimizer: {hyperparams["optimizer"]}')
        print(f'Learning Rate: lr0={hyperparams["lr0"]:.6f}, lrf={hyperparams["lrf"]:.4f}')
        print(f'Momentum: {hyperparams["momentum"]:.4f}, Weight Decay: {hyperparams["weight_decay"]:.6f}')
        print(f'Augmentation: hsv_h={hyperparams["hsv_h"]:.3f}, translate={hyperparams["translate"]:.3f}, mixup={hyperparams["mixup"]:.3f}')
        print(f'Loss Weights: box={hyperparams["box"]:.2f}, cls={hyperparams["cls"]:.2f}, dfl={hyperparams["dfl"]:.2f}')
        print(f'{"=" * 80}')
        
        # Load fresh model for this trial
        trial_model = YOLO(str(model_path))
        
        # Train model with hyperparameters
        results = trial_model.train(
            data=str(DATA_YAML_PATH),
            project=str(trial_dir),
            name='train',
            exist_ok=True,
            **hyperparams
        )
        
        # Validate model
        val_results = trial_model.val(
            data=str(DATA_YAML_PATH),
            split='val',
            verbose=False
        )
        
        # Extract metrics
        map50 = float(val_results.box.map50)  # mAP@0.5 (primary metric)
        map50_95 = float(val_results.box.map)  # mAP@0.5:0.95
        precision = float(val_results.box.mp)  # Mean precision
        recall = float(val_results.box.mr)  # Mean recall
        
        # Log to W&B
        if wandb_run:
            wandb.log({
                'trial_number': trial.number,
                'val/mAP@0.5': map50,
                'val/mAP@0.5:0.95': map50_95,
                'val/precision': precision,
                'val/recall': recall,
            })
            wandb.finish()
        
        # Save trial results
        trial_results = {
            'trial_number': trial.number,
            'hyperparameters': {k: float(v) if isinstance(v, (np.floating, np.integer)) else v 
                              for k, v in hyperparams.items()},
            'metrics': {
                'map50': map50,
                'map50_95': map50_95,
                'precision': precision,
                'recall': recall,
            },
            'timestamp': datetime.now().isoformat()
        }
        
        with open(trial_dir / 'results.json', 'w') as f:
            json.dump(trial_results, f, indent=2)
        
        # Print results
        print(f'\n✓ Trial {trial.number} completed:')
        print(f'  mAP@0.5: {map50:.4f}')
        print(f'  mAP@0.5:0.95: {map50_95:.4f}')
        print(f'  Precision: {precision:.4f}')
        print(f'  Recall: {recall:.4f}')
        
        # Clean up to save disk space
        try:
            weights_dir = trial_dir / 'train' / 'weights'
            if weights_dir.exists():
                shutil.rmtree(weights_dir)
        except Exception as e:
            print(f'  ⚠️  Could not clean up weights: {e}')
        
        return map50
        
    except Exception as e:
        print(f'\n❌ Trial {trial.number} failed with error: {e}')
        import traceback
        traceback.print_exc()
        
        if wandb_run:
            wandb.finish()
        
        # Return very low score for failed trials (not 0 to avoid division issues)
        return 0.001

print('✓ Objective function defined')
print('  Optimization Metric: mAP@0.5 (to maximize)')
print('  Additional Tracking: mAP@0.5:0.95, precision, recall')
print('  Error Handling: Graceful fallback with detailed logging')

## 7. Run Hyperparameter Optimization

In [ ]:
# ============================================================================
# RUN HYPERPARAMETER OPTIMIZATION WITH OPTUNA
# ============================================================================

print('\n' + '=' * 80)
print('STARTING HYPERPARAMETER OPTIMIZATION')
print('=' * 80)
print(f'Model: {MODEL_NAME}')
print(f'Dataset: {YOLO_DATASET_ROOT.name}')
print(f'Number of Trials: {N_TRIALS}')
print(f'Epochs per Trial: {EPOCHS_PER_TRIAL}')
print(f'Timeout: {TIMEOUT_HOURS} hours' if TIMEOUT_HOURS else 'No timeout')
print(f'Device: {device}')
print('=' * 80)

# Create Optuna study
study = optuna.create_study(
    study_name=f'{MODEL_NAME}_optuna_{RUN_TIMESTAMP}',
    direction='maximize',  # Maximize mAP@0.5
    sampler=optuna.samplers.TPESampler(
        seed=42,
        n_startup_trials=N_STARTUP_TRIALS,  # Random trials before optimization
        multivariate=True,  # Consider parameter interactions
        group=True  # Group related parameters
    ),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=N_STARTUP_TRIALS,
        n_warmup_steps=15,  # Wait before pruning
        interval_steps=5  # Check every 5 steps
    )
)

# Run optimization
start_time = datetime.now()
print(f'\n🚀 Optimization started at {start_time.strftime("%Y-%m-%d %H:%M:%S")}')

try:
    study.optimize(
        objective,
        n_trials=N_TRIALS,
        timeout=TIMEOUT_HOURS * 3600 if TIMEOUT_HOURS else None,
        show_progress_bar=True,
        callbacks=[
            lambda study, trial: print(f'\n✓ Completed {len(study.trials)}/{N_TRIALS} trials')
        ]
    )
except KeyboardInterrupt:
    print('\n⚠️  Optimization interrupted by user')
except Exception as e:
    print(f'\n❌ Optimization failed: {e}')
    import traceback
    traceback.print_exc()

end_time = datetime.now()
duration = end_time - start_time

print('\n' + '=' * 80)
print('OPTIMIZATION COMPLETED')
print('=' * 80)
print(f'Started: {start_time.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Ended: {end_time.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Duration: {duration}')
print(f'Total Trials: {len(study.trials)}')
print(f'Completed Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}')
print(f'Pruned Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}')
print(f'Failed Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}')
print(f'\nBest Trial: {study.best_trial.number}')
print(f'Best mAP@0.5: {study.best_value:.4f}')
print('=' * 80)

## 8. Analyze Best Hyperparameters

In [ ]:
# ============================================================================
# EXTRACT AND DISPLAY BEST HYPERPARAMETERS
# ============================================================================

print('\n' + '=' * 80)
print('BEST HYPERPARAMETERS')
print('=' * 80)

best_params = study.best_params
best_trial = study.best_trial

print(f'\nBest Trial Number: {best_trial.number}')
print(f'Best mAP@0.5: {study.best_value:.4f}')
print('\nOptimized Hyperparameters:')
print(json.dumps(best_params, indent=2))

# Save best parameters to JSON
best_params_json = RUN_DIR / 'best_hyperparameters.json'
with open(best_params_json, 'w') as f:
    json.dump({
        'model': MODEL_NAME,
        'dataset': str(YOLO_DATASET_ROOT),
        'best_trial': best_trial.number,
        'best_map50': study.best_value,
        'total_trials': len(study.trials),
        'hyperparameters': best_params,
        'optimization_config': {
            'n_trials': N_TRIALS,
            'epochs_per_trial': EPOCHS_PER_TRIAL,
            'batch_size': BATCH_SIZE,
            'image_size': IMAGE_SIZE,
        },
        'timestamp': datetime.now().isoformat()
    }, f, indent=2)

print(f'\n✓ Best hyperparameters saved to: {best_params_json}')

# Save to YAML format (ready for YOLO training)
best_params_yaml = RUN_DIR / 'best_hparams.yaml'
with open(best_params_yaml, 'w') as f:
    yaml.dump(best_params, f, default_flow_style=False, sort_keys=False)

print(f'✓ Best hyperparameters saved to: {best_params_yaml}')
print('=' * 80)

## 9. Visualize Optimization Results

In [ ]:
# ============================================================================
# VISUALIZE OPTIMIZATION HISTORY
# ============================================================================

print('\n' + '=' * 80)
print('GENERATING OPTIMIZATION VISUALIZATIONS')
print('=' * 80)

# Plot 1: Optimization History
print('\n📈 Creating optimization history plot...')
fig_history = plot_optimization_history(study)
fig_history.update_layout(
    title=f'{MODEL_NAME} - Hyperparameter Optimization History',
    xaxis_title='Trial Number',
    yaxis_title='mAP@0.5',
    template='plotly_white',
    width=1200,
    height=600
)
fig_history.show()

# Save figure
optimization_history_path = RUN_DIR / 'optimization_history.html'
fig_history.write_html(str(optimization_history_path))
print(f'✓ Saved to: {optimization_history_path}')

# Also save as image if kaleido is available
try:
    optimization_history_img = RUN_DIR / 'optimization_history.png'
    fig_history.write_image(str(optimization_history_img), width=1200, height=600, scale=2)
    print(f'✓ Saved to: {optimization_history_img}')
except Exception as e:
    print(f'  ℹ️  Could not save PNG (kaleido not available): {e}')
    optimization_history_img = None

print('=' * 80)

In [ ]:
# ============================================================================
# VISUALIZE PARAMETER IMPORTANCE
# ============================================================================

print('\n📊 Creating parameter importance plot...')

param_importance_path = None
try:
    fig_importance = plot_param_importances(study)
    fig_importance.update_layout(
        title=f'{MODEL_NAME} - Hyperparameter Importance',
        xaxis_title='Importance',
        yaxis_title='Parameter',
        template='plotly_white',
        width=1200,
        height=800
    )
    fig_importance.show()
    
    # Save figure
    param_importance_path = RUN_DIR / 'parameter_importance.html'
    fig_importance.write_html(str(param_importance_path))
    print(f'✓ Saved to: {param_importance_path}')
    
    # Save as image
    try:
        param_importance_img = RUN_DIR / 'parameter_importance.png'
        fig_importance.write_image(str(param_importance_img), width=1200, height=800, scale=2)
        print(f'✓ Saved to: {param_importance_img}')
    except Exception as e:
        print(f'  ℹ️  Could not save PNG: {e}')
        param_importance_img = None
        
except (RuntimeError, ValueError) as e:
    print(f'⚠️  Could not generate parameter importance plot: {e}')
    print('  (This can happen when trials have insufficient data variation)')
    param_importance_img = None

print('=' * 80)

## 10. Visualize Parameter Relationships

In [ ]:
# ============================================================================
# VISUALIZE SLICE PLOTS (PARAMETER RELATIONSHIPS)
# ============================================================================

print('\n📊 Creating parameter slice plots...')

try:
    fig_slice = plot_slice(study)
    fig_slice.update_layout(
        title=f'{MODEL_NAME} - Parameter Slice Plot',
        template='plotly_white',
        width=1400,
        height=1000
    )
    fig_slice.show()
    
    # Save figure
    slice_path = RUN_DIR / 'parameter_slice.html'
    fig_slice.write_html(str(slice_path))
    print(f'✓ Saved to: {slice_path}')
    
    # Save as image
    try:
        slice_img_path = RUN_DIR / 'parameter_slice.png'
        fig_slice.write_image(str(slice_img_path), width=1400, height=1000, scale=2)
        print(f'✓ Saved to: {slice_img_path}')
    except Exception as e:
        print(f'  ℹ️  Could not save PNG: {e}')
        
except Exception as e:
    print(f'⚠️  Could not generate parameter slice plot: {e}')

print('=' * 80)

## 10. Create Results Summary

In [ ]:
# ============================================================================
# CREATE TRIALS SUMMARY
# ============================================================================

print('\n' + '=' * 80)
print('TRIALS SUMMARY')
print('=' * 80)

# Compile all trial data
trials_data = []
for trial in study.trials:
    trial_info = {
        'trial': trial.number,
        'mAP@0.5': trial.value if trial.value else 0.0,
        'state': trial.state.name,
        'duration_seconds': (trial.datetime_complete - trial.datetime_start).total_seconds() if trial.datetime_complete else None,
    }
    # Add all parameters
    trial_info.update(trial.params)
    trials_data.append(trial_info)

# Create DataFrame
df_trials = pd.DataFrame(trials_data)

# Sort by performance
df_trials_sorted = df_trials.sort_values('mAP@0.5', ascending=False)

print('\n📊 TOP 10 TRIALS:')
print('=' * 80)
# Display top 10 with selected columns
display_cols = ['trial', 'mAP@0.5', 'state', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'mixup']
available_cols = [col for col in display_cols if col in df_trials_sorted.columns]
print(df_trials_sorted[available_cols].head(10).to_string(index=False))
print('=' * 80)

# Save complete trials summary
trials_csv_path = RUN_DIR / 'trials_summary.csv'
df_trials_sorted.to_csv(trials_csv_path, index=False)
print(f'\n✓ Complete trials summary saved to: {trials_csv_path}')

# Save study object
study_path = RUN_DIR / 'optuna_study.pkl'
import pickle
with open(study_path, 'wb') as f:
    pickle.dump(study, f)
print(f'✓ Optuna study object saved to: {study_path}')

print('=' * 80)

## 11. Save Hyperparameters for Training

In [ ]:
# ============================================================================
# PREPARE FINAL TRAINING CONFIGURATION
# ============================================================================

print('\n' + '=' * 80)
print('PREPARING FINAL TRAINING CONFIGURATION')
print('=' * 80)

# Create training directory
training_dir = BASE_DIR / 'training'
training_dir.mkdir(exist_ok=True)

# Prepare final training hyperparameters
final_training_params = best_params.copy()
final_training_params.update({
    # Extended training settings
    'epochs': 100,  # Full training epochs
    'batch': BATCH_SIZE,
    'imgsz': IMAGE_SIZE,
    'device': device,
    
    # Training control
    'patience': 25,  # Early stopping patience
    'save': True,  # Save models
    'save_period': 10,  # Save checkpoint every N epochs
    'plots': True,  # Generate training plots
    'verbose': True,  # Detailed output
    
    # Efficiency
    'cache': True,  # Cache images
    'workers': 8,  # Data loading workers
    'amp': True,  # Automatic mixed precision
    
    # Validation
    'val': True,  # Run validation
    
    # Project organization
    'project': str(training_dir / 'runs'),
    'name': f'{MODEL_NAME}_optimized',
    'exist_ok': True,
})

# Save training configuration
training_config_path = training_dir / f'{MODEL_NAME}_optimized_config.yaml'
with open(training_config_path, 'w') as f:
    yaml.dump(final_training_params, f, default_flow_style=False, sort_keys=False)

print(f'\n✓ Training configuration saved to: {training_config_path}')

# Also save as JSON with metadata
training_config_json = training_dir / f'{MODEL_NAME}_optimized_config.json'
with open(training_config_json, 'w') as f:
    json.dump({
        'model': MODEL_NAME,
        'base_model_path': str(model_path),
        'dataset_root': str(YOLO_DATASET_ROOT),
        'data_yaml_path': str(DATA_YAML_PATH),
        'optimization_results': {
            'best_trial': study.best_trial.number,
            'best_map50': study.best_value,
            'total_trials': len(study.trials),
            'optimization_duration': str(duration),
        },
        'hyperparameters': final_training_params,
        'timestamp': datetime.now().isoformat(),
        'notes': 'Use these hyperparameters for full model training with 100 epochs'
    }, f, indent=2)

print(f'✓ Training configuration (with metadata) saved to: {training_config_json}')

print('\n📋 Training Configuration Summary:')
print(f'  Epochs: {final_training_params["epochs"]}')
print(f'  Batch Size: {final_training_params["batch"]}')
print(f'  Image Size: {final_training_params["imgsz"]}')
print(f'  Optimizer: {final_training_params["optimizer"]}')
print(f'  Learning Rate: {final_training_params["lr0"]:.6f}')
print(f'  Device: {final_training_params["device"]}')

print('=' * 80)

## 12. Train Final Model with Optimized Hyperparameters

Now train the final model using the best hyperparameters found during optimization.

In [ ]:
# ============================================================================
# TRAIN FINAL MODEL WITH OPTIMIZED HYPERPARAMETERS
# ============================================================================

print('\n' + '=' * 80)
print('TRAINING FINAL MODEL WITH OPTIMIZED HYPERPARAMETERS')
print('=' * 80)

# Load fresh model for final training
print(f'\n📦 Loading base model: {MODEL_NAME}')
final_model = YOLO(str(model_path))

print(f'\n🚀 Starting final training with best hyperparameters...')
print(f'  Epochs: {final_training_params["epochs"]}')
print(f'  Dataset: {DATA_YAML_PATH}')
print(f'  Device: {device}')
print('\nThis may take a while. Training progress will be displayed below.')
print('=' * 80)

# Train model with optimized hyperparameters
final_results = final_model.train(
    data=str(DATA_YAML_PATH),
    **final_training_params
)

print('\n' + '=' * 80)
print('FINAL TRAINING COMPLETED')
print('=' * 80)

# Get final validation metrics
final_val_results = final_model.val(data=str(DATA_YAML_PATH))

final_metrics = {
    'map50': float(final_val_results.box.map50),
    'map50_95': float(final_val_results.box.map),
    'precision': float(final_val_results.box.mp),
    'recall': float(final_val_results.box.mr),
}

print('\n📊 Final Model Performance:')
print(f'  mAP@0.5: {final_metrics["map50"]:.4f}')
print(f'  mAP@0.5:0.95: {final_metrics["map50_95"]:.4f}')
print(f'  Precision: {final_metrics["precision"]:.4f}')
print(f'  Recall: {final_metrics["recall"]:.4f}')

# Compare with optimization results
improvement = final_metrics['map50'] - study.best_value
print(f'\n📈 Improvement over trial performance: {improvement:+.4f}')
print(f'  Trial best: {study.best_value:.4f}')
print(f'  Final model: {final_metrics["map50"]:.4f}')

print('=' * 80)

## 13. Save Final Model

In [ ]:
# ============================================================================
# SAVE FINAL OPTIMIZED MODEL
# ============================================================================

print('\n' + '=' * 80)
print('SAVING FINAL OPTIMIZED MODEL')
print('=' * 80)

# Generate model filename with timestamp
model_timestamp = datetime.now().strftime('%Y%m%d')
final_model_name = f'{MODEL_NAME}_optimized_{model_timestamp}.pt'
final_model_path = MODELS_DIR / final_model_name

# Copy best weights to models directory
weights_path = training_dir / 'runs' / f'{MODEL_NAME}_optimized' / 'weights' / 'best.pt'
if weights_path.exists():
    shutil.copy(weights_path, final_model_path)
    print(f'\n✓ Final model saved to: {final_model_path}')
    print(f'  Size: {final_model_path.stat().st_size / (1024*1024):.1f} MB')
else:
    print(f'\n⚠️  Best weights not found at: {weights_path}')
    print('  Model may still be in training directory')

# Save model metadata
metadata = {
    'model_name': MODEL_NAME,
    'model_path': str(final_model_path),
    'dataset': str(YOLO_DATASET_ROOT),
    'training_date': datetime.now().isoformat(),
    'optimization': {
        'n_trials': len(study.trials),
        'best_trial': study.best_trial.number,
        'trial_map50': study.best_value,
        'optimization_duration': str(duration),
    },
    'hyperparameters': best_params,
    'final_metrics': final_metrics,
    'training_config': {
        'epochs': final_training_params['epochs'],
        'batch_size': final_training_params['batch'],
        'image_size': final_training_params['imgsz'],
    }
}

metadata_path = MODELS_DIR / f'{MODEL_NAME}_optimized_{model_timestamp}_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f'✓ Model metadata saved to: {metadata_path}')
print('=' * 80)

## 12. Generate PDF Report

Create a comprehensive PDF report with optimization results, visualizations, and model performance.

In [ ]:
# ============================================================================
# GENERATE PDF REPORT
# ============================================================================

from reportlab.lib.pagesizes import A4
from reportlab.lib import colors as rl_colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from PIL import Image as PILImage

print('\n' + '=' * 80)
print('GENERATING PDF REPORT')
print('=' * 80)

# Create PDF report
pdf_report_path = RUN_DIR / f'{MODEL_NAME}_hyperparameter_tuning_report.pdf'

doc = SimpleDocTemplate(str(pdf_report_path), pagesize=A4,
                       rightMargin=30, leftMargin=30,
                       topMargin=30, bottomMargin=30)

story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=24,
    textColor=rl_colors.HexColor('#2c3e50'),
    spaceAfter=30,
    alignment=TA_CENTER
)

heading_style = ParagraphStyle(
    'CustomHeading',
    parent=styles['Heading2'],
    fontSize=16,
    textColor=rl_colors.HexColor('#34495e'),
    spaceAfter=12,
    spaceBefore=20
)

# Title
story.append(Paragraph(f'{MODEL_NAME} Hyperparameter Tuning Report', title_style))
story.append(Spacer(1, 12))

# Configuration info
info_data = [
    ['Model:', MODEL_NAME],
    ['Dataset:', YOLO_DATASET_ROOT.name],
    ['Timestamp:', datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
    ['Total Trials:', str(len(study.trials))],
    ['Best Trial:', str(study.best_trial.number)],
    ['Best mAP@0.5:', f'{study.best_value:.4f}']
]

info_table = Table(info_data, colWidths=[2.2*inch, 3.8*inch])
info_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, -1), rl_colors.HexColor('#ecf0f1')),
    ('TEXTCOLOR', (0, 0), (-1, -1), rl_colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.white)
]))
story.append(info_table)
story.append(Spacer(1, 20))

# Best hyperparameters
story.append(PageBreak())
story.append(Paragraph('Best Hyperparameters', heading_style))

hyperparam_data = [['Parameter', 'Value']]
for key, value in best_params.items():
    hyperparam_data.append([key, f'{value:.6f}' if isinstance(value, float) else str(value)])

hyperparam_table = Table(hyperparam_data, colWidths=[3*inch, 3*inch])
hyperparam_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 12),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(hyperparam_table)
story.append(Spacer(1, 20))

# Top 10 trials
story.append(PageBreak())
story.append(Paragraph('Top 10 Trials', heading_style))

top10_data = [['Trial', 'mAP@0.5', 'State']]
for _, row in df_trials_sorted.head(10).iterrows():
    top10_data.append([
        str(int(row['trial'])),
        f"{row['mAP@0.5']:.4f}",
        row['state']
    ])

top10_table = Table(top10_data, colWidths=[1.5*inch, 2*inch, 2.5*inch])
top10_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#27ae60')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(top10_table)
story.append(Spacer(1, 20))

# Optimization history
story.append(PageBreak())
story.append(Paragraph('Optimization History', heading_style))
story.append(Spacer(1, 12))

optimization_history_img = RUN_DIR / 'optimization_history.png'
if optimization_history_img.exists():
    try:
        with PILImage.open(optimization_history_img) as img:
            img_width, img_height = img.size
            aspect_ratio = img_height / img_width
            pdf_width = 6.5 * inch
            pdf_height = pdf_width * aspect_ratio
            if pdf_height > 7 * inch:
                pdf_height = 7 * inch
                pdf_width = pdf_height / aspect_ratio
            story.append(Image(str(optimization_history_img), width=pdf_width, height=pdf_height))
    except Exception as e:
        print(f'⚠️  Could not load optimization history: {e}')
        story.append(Paragraph('Optimization history chart not available.', styles['Normal']))
else:
    story.append(Paragraph('Optimization history chart not found (PNG format required).', styles['Normal']))

story.append(Spacer(1, 20))

# Parameter importance
story.append(PageBreak())
story.append(Paragraph('Parameter Importance', heading_style))
story.append(Spacer(1, 12))

param_importance_img = RUN_DIR / 'parameter_importance.png'
if param_importance_img.exists():
    try:
        with PILImage.open(param_importance_img) as img:
            img_width, img_height = img.size
            aspect_ratio = img_height / img_width
            pdf_width = 6.5 * inch
            pdf_height = pdf_width * aspect_ratio
            if pdf_height > 7 * inch:
                pdf_height = 7 * inch
                pdf_width = pdf_height / aspect_ratio
            story.append(Image(str(param_importance_img), width=pdf_width, height=pdf_height))
    except Exception as e:
        print(f'⚠️  Could not load parameter importance: {e}')
        story.append(Paragraph('Parameter importance chart not available.', styles['Normal']))
else:
    story.append(Paragraph('Parameter importance chart not available or could not be generated.', styles['Normal']))

story.append(Spacer(1, 20))

# Final model performance (if available)
if 'final_metrics' in globals():
    story.append(PageBreak())
    story.append(Paragraph('Final Model Performance', heading_style))
    
    final_perf_data = [
        ['Metric', 'Value'],
        ['mAP@0.5', f"{final_metrics['map50']:.4f}"],
        ['mAP@0.5:0.95', f"{final_metrics['map50_95']:.4f}"],
        ['Precision', f"{final_metrics['precision']:.4f}"],
        ['Recall', f"{final_metrics['recall']:.4f}"],
    ]
    
    final_perf_table = Table(final_perf_data, colWidths=[3*inch, 3*inch])
    final_perf_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#e74c3c')),
        ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('FONTSIZE', (0, 1), (-1, -1), 11),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
        ('TOPPADDING', (0, 0), (-1, -1), 8),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
        ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
    ]))
    story.append(final_perf_table)
    story.append(Spacer(1, 20))

# Footer
story.append(Spacer(1, 30))
story.append(Paragraph('Generated by YOLO Hyperparameter Tuning Notebook',
                      ParagraphStyle('Footer', parent=styles['Normal'],
                                   alignment=TA_CENTER, textColor=rl_colors.grey)))
story.append(Paragraph('BDD100K Dataset - Computer Vision Project',
                      ParagraphStyle('Footer2', parent=styles['Normal'],
                                   alignment=TA_CENTER, textColor=rl_colors.grey)))

# Build PDF
try:
    doc.build(story)
    print(f'\n✓ PDF report generated: {pdf_report_path}')
    print(f'  Size: {pdf_report_path.stat().st_size / 1024:.2f} KB')
except Exception as e:
    print(f'\n❌ Error generating PDF: {e}')
    import traceback
    traceback.print_exc()

print('=' * 80)

In [ ]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print('\n\n')
print('=' * 80)
print('HYPERPARAMETER OPTIMIZATION COMPLETE!')
print('=' * 80)

print(f'\n📊 Project: {MODEL_NAME} on {YOLO_DATASET_ROOT.name}')
print(f'📅 Date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

print(f'\n🔬 Optimization Summary:')
print(f'  Total Trials: {len(study.trials)}')
print(f'  Completed: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}')
print(f'  Best Trial: {study.best_trial.number}')
print(f'  Best Trial mAP@0.5: {study.best_value:.4f}')
print(f'  Duration: {duration}')

if 'final_metrics' in globals():
    print(f'\n🎯 Final Model Performance:')
    print(f'  mAP@0.5: {final_metrics["map50"]:.4f}')
    print(f'  mAP@0.5:0.95: {final_metrics["map50_95"]:.4f}')
    print(f'  Precision: {final_metrics["precision"]:.4f}')
    print(f'  Recall: {final_metrics["recall"]:.4f}')

print(f'\n📁 Generated Files:')
print(f'  📊 Optimization Results:')
print(f'    - {RUN_DIR / "best_hyperparameters.json"}')
print(f'    - {RUN_DIR / "best_hparams.yaml"}')
print(f'    - {RUN_DIR / "trials_summary.csv"}')
print(f'    - {RUN_DIR / "optuna_study.pkl"}')
print(f'  📈 Visualizations:')
print(f'    - {RUN_DIR / "optimization_history.html"}')
print(f'    - {RUN_DIR / "parameter_importance.html"}')
print(f'    - {RUN_DIR / "parameter_slice.html"}')
print(f'  📄 PDF Report:')
print(f'    - {RUN_DIR / f"{MODEL_NAME}_hyperparameter_tuning_report.pdf"}')

if 'final_model_path' in globals():
    print(f'  🎯 Final Model:')
    print(f'    - {final_model_path}')
    print(f'    - {metadata_path}')
    print(f'  ⚙️  Training Config:')
    print(f'    - {training_config_path}')
    print(f'    - {training_config_json}')

print(f'\n📂 All results saved to: {RUN_DIR}')

print(f'\n🎓 Top 5 Hyperparameters (by importance):')
try:
    importances = optuna.importance.get_param_importances(study)
    for i, (param, importance) in enumerate(list(importances.items())[:5], 1):
        print(f'  {i}. {param}: {importance:.4f}')
except:
    print('  (Not available - requires completed trials with variation)')

print(f'\n🚀 Next Steps:')
print(f'  1. Review PDF report: {RUN_DIR / f"{MODEL_NAME}_hyperparameter_tuning_report.pdf"}')
print(f'  2. Review optimization visualizations in: {RUN_DIR}')
if 'final_model_path' in globals():
    print(f'  3. Use final model for inference: {final_model_path}')
    print(f'  4. Check training plots in: {training_dir / "runs" / f"{MODEL_NAME}_optimized"}')
else:
    print(f'  3. Run final training section to create optimized model')
print(f'  5. Consider testing different model sizes (yolov8s, yolov8m, etc.)')
print(f'  6. Evaluate on test set for final performance metrics')

print('\n' + '=' * 80)
print('SUCCESS! ✓')
print('=' * 80)